# IE 306.02 SimPy Assingment 1
## Group 24
- Özge Dinçsoy
- Alperen Yakut
- Mustafa Enes Çakır

**NOTE: We write required statistics programatically, so you need to do "Cell->Run All" (or run all cells from top to bottom manuelly) in order to see correct results.**

**Requirements:** Python 3.6

In [ ]:
import simpy
import random

**Define a set of globals that define the characteristics of the model instance to be simulated.**
- *RANDOM_SEED* **->** The seed for the random number generators
- *INTERARRIVAL_MEAN* **->** The mean of exponentially distributed arrival time
- *INTERARRIVAL_RATE* **->** The rate of exponentially distributed arrival time
- *SERVICE_TIME_OPERATOR_MEAN* **->** The mean of Gaussian distributed operator service time
- *SERVICE_TIME_OPERATOR_DEVIATION* **->** The standard deviation of Gaussian distributed operator service time
- *SERVICE_TIME_EXPERT_MEAN* **->** The mean of Exponentially distributed expert service time
- *SERVICE_TIME_EXPERT_DEVIATION* **->** The standard deviation of Normally distributed expert service time
- *BREAK_TIME* **->** The length of expert's break
- *BREAK_TIME_RATE* **->** The mean of Poisson distributed expert's break time. 8 breaks for 8 hour shift, so 1 break for 1 hour. 1/60 break for 1 minutes

In [ ]:
RANDOM_SEED = 9780
INTERARRIVAL_MEAN = 15.6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

SERVICE_TIME_OPERATOR_MEAN = 4.2
SERVICE_TIME_OPERATOR_DEVIATION = 1.5

SERVICE_TIME_EXPERT_MEAN = 9.3
SERVICE_TIME_EXPERT_DEVIATION = 3.1
SERVICE_TIME_EXPERT_RATE = 1.0 / SERVICE_TIME_EXPERT_MEAN

BREAK_TIME = 5
BREAK_TIME_RATE = 1/60
CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

**Define the necessary set of arrays for statistics**

In [ ]:
# Duration of the conversation between the customer and the operator (Operator service time)
service_times_operator = []

# Duration of the conversation between the customer and the expert (Expert service time)
service_times_expert = []

# Time spent by a customer while it waits for the operator (Operator Queue waiting time)
queue_w_times_operator = []

# Time spent by a customer while it waits for the expert (Expert Queue waiting time)
queue_w_times_expert = []

# Simulation end time
end_time = 0;

**Explanation of the process**

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the operator resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

* When the customer done with operator, he/she is registered as a request to the expert resource. The customer is put on hold until the resource activates it back. 

* When the expert resource is available, the customer is activated and it then initiates the get_help process. The duration of a help session is determined randomly according to a exponentially distribution.

In [ ]:
class Customer(object):
    def __init__(self, name, env, operator, expert):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))

        with operator.request() as req:
            yield req
            print('%s is assigned to a operator at %g' % (self.name, self.env.now))
            queue_w_times_operator.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done operator at %g' % (self.name, self.env.now))
            self.done_t = self.env.now
    
        with expert.request() as req:
            yield req
            print('%s is assigned to a expert at %g' % (self.name, self.env.now))
            queue_w_times_expert.append(self.env.now - self.done_t)
            yield self.env.process(self.get_help())
            print('%s is done expert at %g' % (self.name, self.env.now))
            global end_time 
            end_time = self.env.now

    def ask_question(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR_MEAN, SERVICE_TIME_OPERATOR_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR_MEAN, SERVICE_TIME_OPERATOR_DEVIATION)

        yield self.env.timeout(duration)
        service_times_operator.append(duration)
        
    def get_help(self):
        duration = random.expovariate(SERVICE_TIME_EXPERT_RATE)
        while duration < 0:
            duration = random.expovariate(SERVICE_TIME_EXPERT_RATE)
            
        yield self.env.timeout(duration)
        service_times_expert.append(duration)


In [ ]:
class CustomerNorm(Customer):
    """ Customer that has normally distributed expert service"""
    def get_help(self):
        duration = random.gauss(SERVICE_TIME_EXPERT_MEAN, SERVICE_TIME_EXPERT_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_EXPERT_MEAN, SERVICE_TIME_EXPERT_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_expert.append(duration)



In [ ]:
def customer_generator(env, operator, expert):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator, expert)  

In [ ]:
def customer_norm_generator(env, operator, expert):
    """Generate new customer with normally distributed expert service that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = CustomerNorm('Customer %s' %(i+1), env, operator, expert)  

In [ ]:
def break_generator(env, operator, expert):
    """Generate new break for expert."""
    while True:
        if len(queue_w_times_expert) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with expert.request() as req:
            yield req
            print('Expert gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Expert break done at %g' % (env.now))



## Expert service time Exponentially distributed with a mean of 9.3 - 1000 CUSTOMER

In [ ]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

In [ ]:
# Calculating statistics
CUSTOMER_COUNT = 1000
print("Expert service time Exponentially distributed with a mean of 9.3 - 1000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))


## Expert service time Exponentially distributed with a mean of 9.3 - 5000 CUSTOMER

In [ ]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 5000

In [ ]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

In [ ]:
# Calculating statistics
CUSTOMER_COUNT = 5000
print("Expert service time Exponentially distributed with a mean of 9.3 - 5000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))

## Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 1000 Customer

In [ ]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 1000

In [ ]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_norm_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

In [ ]:
# Calculating statistics

CUSTOMER_COUNT = 1000
print("Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 1000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))

## Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 5000 Customer

In [ ]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 5000

In [ ]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_norm_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

In [ ]:
# Calculating statistics

CUSTOMER_COUNT = 5000
print("Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 5000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))